In [1]:
import os
from enum import Enum
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mycolorpy import colorlist as mcp
from z_base import Experiment, Mols, T_ZERO
from scipy.stats import linregress

plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('font', size=20)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=15)
plt.rc('legend', fontsize=15)
plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams['figure.subplot.left'] = 0.1
plt.rcParams['figure.subplot.right'] = 0.99
plt.rcParams['figure.subplot.top'] = 0.99
plt.rcParams['figure.subplot.bottom'] = 0.1


In [2]:
%matplotlib qt

In [23]:
DF=pd.read_excel(r'Datas\All_viscosity.xlsx',index_col=0)

In [4]:
class W(Enum):
    w = "w mol  %"
    w_mass = "w mass  %"
    w_vol = "w vol  %"
    T = 'Temperature  C'
    Nu = 'Viscosity  cP'
    ln_Nu = 'log Viscosity  cP'
    V = 'Kinematic viscosity cCt'
    ln_D = 'log Diffusion'
    D = 'Diffusion'
    T_1 = '1000/Temperature  K$^{-1}$'

In [12]:
# Comparation pure
x_i = W.T
y_i = W.ln_D

compound_filter = (DF["w"] == 100) | (DF["w"] == 0)
df =DF[compound_filter]
data = []

fig, ax = plt.subplots(num="DT")
for (compound, *_), group in df.groupby(by=['compound']):
    y = group[y_i.name]
    T = group[x_i.name]
    ax.scatter(T, y, s=5,label=f"{compound:5}")

    # R = linregress(T, y)
    # line_x = np.array([0.00315, 0.0035])
    # ax.plot(
    #     line_x*1000,
    #     R.slope * line_x + R.intercept,
    #     linewidth=1,
    #     label=f"{compound:5}",
    # )
ax.set_xlabel(x_i.value)
ax.set_ylabel(y_i.value)

plt.legend()
plt.show()

In [6]:
# Grouped compound Y(x)
compound = Mols.ocm
x_i = W.T_1
y_i = W.ln_Nu
g_i = W.w_mass

compound_filter = (DF["compound"] == compound.name)
# | (DF["compound"] == Mols.butanol.name)


fig, ax = plt.subplots(num="DT")
grouped = DF[compound_filter].groupby(by=[g_i.name])
colors_list = mcp.gen_color(cmap="brg", n=len(grouped))
for _, group in grouped:
    y = group[y_i.name]
    T = group[x_i.name]
    color = colors_list.pop()

    ax.scatter(T*1000, y, c=color, s=2)

    R = linregress(T, y)
    line_x = np.array([0.00315, 0.0035])
    ax.plot(
        line_x*1000,
        R.slope * line_x + R.intercept,
        c=color,
        linewidth=1,
        label=f"{group[g_i.name].mean(): 5.2f} %",
    )
# ax.set_title(compound.name.upper())
# ax.set_xlim(15, 40)
ax.set_xlabel(x_i.value)
ax.set_ylabel(y_i.value)

plt.legend()
plt.show()

In [10]:
# Window Y(x)
compound = Mols.ocm
x_i = W.w
y_i = W.ln_D

compound_filter = (DF["compound"] == compound.name)|(DF["compound"] == Mols.butanol.name)
# | (DF["compound"] == Mols.butanol.name)
grouped = DF[compound_filter]
temper_list = np.arange(15, 41, 5)+T_ZERO
colors_list = mcp.gen_color(cmap="jet", n=len(temper_list))

fig, ax = plt.subplots(num="DX")
for T in temper_list:
    cond = (T - 2.5 <= grouped["T"]) & (grouped["T"] <= T + 2.5)
    group2= grouped[cond]
    group = group2[[x_i.name, y_i.name]].groupby(by=x_i.name).mean().reset_index()

    color = colors_list.pop()
    x = group[x_i.name]
    y = group[y_i.name]
    ax.scatter(x, y, c=color, label=f"{T-T_ZERO: 3.0f} C")
    line_x=np.array([0,60])
    r=linregress(x[:-1],y[:-1])
    ax.plot(line_x,r.slope*line_x+r.intercept,c=color,)

# ax.set_title(f"{compound.value.upper()}")
ax.set_xlabel(f"{x_i.value} [%]")
ax.set_ylabel(y_i.value)
plt.legend()
plt.show()

In [16]:
# E
x_i = W.w
y_i = W.ln_Nu
shift = 17.24

fig, ax = plt.subplots(num="E")
for (compound, *_), group in DF.groupby(by=["compound"]):
    ys = []
    xs = []
    for (x_val, *_), sub_group in group.groupby(by=[x_i.name]):
        x = np.array(sub_group['T_1'])
        y = np.array(sub_group[y_i.name])
        y = np.log(10**y)
        r = linregress(x, y)

        ys.append(r.slope * 8.314 / 1000)
        xs.append(x_val)
    ax.plot([xs[0], 100], [ys[0], shift])
    ax.scatter(xs, ys, label=f"{compound}")

# ax.set_title(f"E {y_i.value}")
ax.set_xlabel(f"{x_i.value}")
ax.set_ylabel("E  kJ/mol")
plt.legend()
plt.show()

In [28]:
DF[DF['compound']=='peta']

,compound,T,Nu,rho,w_mass,w_vol,w,C_but,C_comp,V,ln_Nu,T_1,D,ln_D
3406,peta,295.25,1171.428571,1.20,0.0,0.000000,0.000000,0.000000,4.022931,976.190476,7.065978,0.003387,0.252043,-1.378157
3407,peta,295.65,1132.000000,1.20,0.0,0.000000,0.000000,0.000000,4.022931,943.333333,7.031741,0.003382,0.261175,-1.342565
3408,peta,295.95,1099.000000,1.20,0.0,0.000000,0.000000,0.000000,4.022931,915.833333,7.002156,0.003379,0.269290,-1.311965
3409,peta,296.45,1044.000000,1.20,0.0,0.000000,0.000000,0.000000,4.022931,870.000000,6.950814,0.003373,0.283956,-1.258936
3410,peta,296.55,1031.166667,1.20,0.0,0.000000,0.000000,0.000000,4.022931,859.305556,6.938439,0.003372,0.287587,-1.246230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4514,peta,312.75,85.400000,1.18,5.8,8.278899,19.858185,0.923368,3.726441,72.372881,4.447335,0.003197,3.662178,1.298058
4515,peta,312.85,85.000000,1.18,5.8,8.278899,19.858185,0.923368,3.726441,72.033898,4.442651,0.003196,3.680588,1.303073
4516,peta,312.95,82.450000,1.18,5.8,8.278899,19.858185,0.923368,3.726441,69.872881,4.412015,0.003195,3.795634,1.333851
4517,peta,313.05,83.800000,1.18,5.8,8.278899,19.858185,0.923368,3.726441,71.016949,4.428433,0.003194,3.735680,1.317930


In [25]:
# Table
g_i = W.w_mass

x_i = W.T_1
y_i = W.ln_Nu

y2_i = W.ln_D
data = []
for (compound, *_), group in DF.groupby(by=["compound"]):
    for (x_val, *_), sub_group in group.groupby(by=[g_i.name]):
        T = sub_group[x_i.name]
        y = sub_group[y_i.name].array
        r = linregress(T, y)
        E_vis = r.slope * 8.314 / 1000
        v0 = np.exp(r.intercept)

        rd = linregress(sub_group[[x_i.name, y2_i.name]])
        E_dif = -rd.slope * 8.314 / 1000
        d0 = np.exp(rd.intercept)
        rho = sub_group['rho'].mean()
        data.append(
            {
                'compound': compound,
                'rho [g/ml]': round(rho, 2),
                'E vis [kJ/mol]': round(E_vis, 2),
                'Vis 0 [cP]': f'{v0: 5.2e}',
                'E dif [kJ/mol]': round(E_dif, 2),
                'D 0 [m2/s]': f'{d0: 5.2e}',
                'x mol': round(x_val, 1),
            }
        )

df = pd.DataFrame(data)
df

,compound,rho [g/ml],E vis [kJ/mol],Vis 0 [cP],E dif [kJ/mol],D 0 [m2/s],x mol
0,butanol,0.83,17.23,2.70e-03,19.72,3.01e+05,100.0
1,dmeg,1.07,18.60,2.29e-03,21.09,3.56e+05,0.0
2,dmeg,1.07,17.48,3.19e-03,19.98,2.57e+05,4.8
3,dmeg,1.05,16.49,4.14e-03,18.98,1.97e+05,9.7
4,dmeg,1.03,17.34,3.12e-03,19.83,2.61e+05,16.8
5,dmeg,1.02,16.29,4.27e-03,18.80,1.92e+05,19.8
6,ocm,1.72,56.02,1.99e-08,58.50,4.08e+10,0.0
7,ocm,1.58,50.44,1.36e-07,52.95,6.03e+09,2.4
8,ocm,1.57,48.64,2.20e-07,51.13,3.71e+09,5.0
9,ocm,1.55,44.79,7.89e-07,47.29,1.03e+09,7.6


In [ ]:
# Density
x_i=W.w_vol
grouped = DF.groupby(by=["compound"])
fig, ax = plt.subplots()
for (compound,*_), group in grouped:
    name = group["compound"].unique()[0]
    group = group[["rho", x_i.name]].groupby(by=[x_i.name]).mean().reset_index()

    x = group[x_i.name]
    y = group["rho"]

    line_x= np.array(x.to_list()+[100.1])
    line_y = np.array(y.to_list()+[0.83])
    r= linregress(line_x,line_y)
    ax.scatter(x, y, label=f"{compound}")
    ax.plot(line_x,line_x*r.slope+r.intercept)
ax.set_title(f"Rho - {x_i.value}")
ax.set_xlabel(f"{x_i.value} [%]")
ax.set_ylabel("Density [g/ml]")
plt.legend()
plt.show()

In [ ]:
## Grouped compounds Y(x)
# x_i = W.T_1
# y_i = W.ln_Nu
# g_i = W.w_mass

# compound_filter = (DF["x"] == 100) | (DF["x"] == 0)
# data=[]

# fig, ax = plt.subplots(num="DT")
# grouped = DF[compound_filter].groupby(by=['compound'])
# # colors_list = mcp.gen_color(cmap="turbo", n=len(grouped))
# for (compound, *_), group in grouped:
#     y = group[y_i.name]
#     T = group[x_i.name]
#     # color = colors_list.pop()

#     ax.scatter(
#         T,
#         y,
#         s=5,
#         # c=color,
#     )

#     R = linregress(T, y)
#     line_x = np.array([0.00315, 0.0035])
#     ax.plot(
#         line_x,
#         R.slope * line_x + R.intercept,
#         # c=color,
#         linewidth=1,
#         label=f"{compound:5} { R.slope: 5.2e}",
#     )
#     data.append(
#             {
#                 'compound': compound,
#                 'x mol': round(x_val,1),
#                 'E vis [kJ/mol]': round(E_vis, 2),
#                 'E dif [kJ/mol]': round(E_dif, 2),
#                 'Vis 0 [cP]': f'{v0: 5.2e}',
#             }
#         )
# # ax.set_title(compound.upper())
# ax.set_xlabel(x_i.value)
# ax.set_ylabel(y_i.value)

# plt.legend()
# plt.show()